In [1]:
import pandas as pd
import numpy as np
import math


In [2]:
# Importando .csv
df = pd.read_csv('dataset/train_data.csv')

# Criando Matrix User X Movie
df_u_m = df.pivot(index='user_id', columns='movie_id', values='rating')

In [3]:
# Calculando correlação de pearson
sim_i_v = df_u_m.corr(method='pearson')

In [8]:
# Implementando predição
def predict(u, i, k = 5):
    sim_items = sim_i_v[i][~sim_i_v[i].isna()].sort_values(ascending=False).index
    rated_items = df_u_m.T[u][df_u_m.T[u] > 0].index
    sim_k = np.intersect1d(sim_items, rated_items)
    top_k = [x for x in sim_items if x in sim_k][:k]
    sum_sim = 0
    dem = 0
    mean_i = df_u_m[i].mean(skipna = True)
    for v in top_k:
        dem += sim_i_v[i][v] * df_u_m[v][u] # - df_u_m[v].mean(skipna = True))
        sum_sim += sim_i_v[i][v]
    if sum_sim == 0:
        return 0
    return dem/sum_sim

In [5]:
# Avaliando modelo
def rmse(test):
    sum_err = 0
    for t in test:
        u = t[0]
        i = t[1]
        r_ui = t[2]
        pred = predict(u, i)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [9]:
rmse(df.values[:1000])

0.6550000500615237